In [3]:
import pandas as pd
import csv
import os

In [2]:
#Activate the first option if whole column width is necessary and the second to make all columns visible
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', 999)

## I. Import the dataset and filter the relevant columns

In [4]:
#Get the report and store it into a dataframe. Check what columns are included in the file
report = pd.read_csv('defects_consolidate_final.csv', dtype = str)
report.columns

Index(['Unnamed: 0', 'enquiry_number', 'job_number', 'enquiry_time',
       'service_code', 'subject_code', 'subject_name', 'priority_code',
       'enq_status_code', 'log_effective_date', 'enquiry_desc', 'officer_code',
       'enq_status_name', 'claim_date', 'claim_tot', 'claim_code', 'area_name',
       'site_address', 'quad', 'feature_location', 'asset_id'],
      dtype='object')

In [5]:
report.head(5)

,Unnamed: 0,enquiry_number,job_number,enquiry_time,service_code,subject_code,subject_name,priority_code,enq_status_code,log_effective_date,...,officer_code,enq_status_name,claim_date,claim_tot,claim_code,area_name,site_address,quad,feature_location,asset_id
0,1,1558145,946775,05/05/2010 09:16,CW,PH1,Urgent Pothole 24 Hr Repair,H24,430,07/05/2010 11:45,...,LW2,0430-Ref'd to 3rd Party by HCC,17/05/2010 09:30,41.67,F1/946775,Tring Town,A41 Tring Bypass,SW,County boundary to B4635 overbridge,A41/011
1,2,1533019,922316,09/02/2010 15:23,WM,RICE,Ice And Snow On Road,H2,400,20/12/2009 12:55,...,ECR2,0400-Repaired/Replaced,17/02/2010 09:13,192.29,F1/922316,Kings Langley,Abbots Rise,SW,Hempstead Road to house 59,2U415/10
2,3,1532872,922185,09/02/2010 11:54,WM,RICE,Ice And Snow On Road,H2,400,20/12/2009 10:10,...,ECR2,0400-Repaired/Replaced,17/02/2010 08:59,195.24,F1/922185,Kings Langley,Abbots Rise,SW,Hempstead Road to house 59,2U415/10
3,4,1350964,646550,05/02/2008 16:41,CW,CPTH,Pothole In Road,NaN,400,06/02/2008 12:00,...,CSC3,0400-Repaired/Replaced,NaN,NaN,NaN,Kings Langley,Abbots Rise,EH,Hempstead Road to house 59,2U415/10
4,5,1377901,701625,10/07/2008 12:00,CW,CPTH,Pothole In Road,NaN,410,10/07/2008 14:40,...,WEB,0410-Fault Made Safe,NaN,NaN,NaN,Kings Langley,Abbots Rise,WEB,Hempstead Road to house 59,2U415/10


In [6]:
#Drop the irrelevant attributes, validated by an SME, and remove whitespaces
report.drop(columns = ['Unnamed: 0', 'job_number', 'subject_name', 
       'enq_status_name', 'area_name'], inplace = True ) 

In [7]:
#Remove all whitespaces from column names
report.columns = [x.replace(" ", "") for x in report.columns]

In [8]:
#Validate with the SME that all the remaining columns are relevant. Done
report.columns

Index(['enquiry_number', 'enquiry_time', 'service_code', 'subject_code',
       'priority_code', 'enq_status_code', 'log_effective_date',
       'enquiry_desc', 'officer_code', 'claim_date', 'claim_tot', 'claim_code',
       'site_address', 'quad', 'feature_location', 'asset_id'],
      dtype='object')

## II. Identify the key feature and format it to avoid empty cells and multiple values per row

In [9]:
#Drop all rows that are empty and drop all duplicate rows
report.dropna(axis=0, how='all', inplace=True)
report.drop_duplicates(inplace=True)

In [10]:
#Don't use since it might crash the notebook
'''#Identify the key feature and split the cell if there are several values within one row
s = report['asset_id'].str.split(';').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'asset_id'
del report['asset_id']
report_mod = report.join(s)
report_mod = report_mod.reset_index(drop=True)
report_mod.shape #Shape after the filtering
'''
report_mod = report
report_mod.shape

(1114171, 16)

In [11]:
#Count the number of cells that are null within the key feature ****CHECK IF THEY CAN BE FILLED WITH SITE NAME***
report_mod['asset_id'].isnull().sum()

510642

In [12]:
#Delete the null rows for the key feature and test it. The result of this sum operation must be 0.
report_mod.dropna(axis=0, how='any', subset = ['asset_id'], inplace = True)
report_mod['asset_id'].isnull().sum()

0

In [13]:
report_mod.shape

(603529, 16)

## 3. Filter the possible values for the relevant attributes

#### Service code

In [14]:
#Check the possible values for the relevant features and their count and sort them:
report_mod.groupby(['service_code'],as_index = False).size().sort_values(ascending = False)

service_code
SL1             148029
CW              137300
SL              122552
CW1              52712
FV               32282
TV1              18015
VG               16049
FL               11568
FV1              11327
OB1              10166
FL1               8844
OB                6280
HD                4723
VXO               4300
SG                3941
AN                2840
FP                2614
FN1               2540
WM                1636
AN1               1625
SG1               1018
SC1                568
TS                 550
BU1                439
GF1                429
GF                 301
NR                 265
BG1                261
WM1                141
MI                  83
UT1                 52
PROW                31
BG                  14
SC                  11
RW1                  8
TR                   7
BS                   7
service_code         1
dtype: int64

In [15]:
#Drop the instances that are out of scope, categorized by feature
filter_list = ['SL1','SL','FV','VG','TV1','FV1','OB1','OB','VXO',
               'SG','AN','FP','FN1','AN1','SG1','SC1','TS','BU1',
              'GF1','GF','MI','UT1','PROW','SC','RW1','BS','TR', 'service_code']
report_mod = report_mod[~report_mod.service_code.isin(filter_list)]
report_mod = report_mod.reset_index(drop=True)

In [16]:
#Check the remaining categories
report_mod.groupby(['service_code'],as_index = False).size().sort_values(ascending = False)

service_code
CW     137300
CW1     52712
FL      11568
FL1      8844
HD       4723
WM       1636
NR        265
BG1       261
WM1       141
BG         14
dtype: int64

In [17]:
report_mod.shape

(217464, 16)

#### Subject Code

In [18]:
#Check the main subject codes and evaluate their relevance. The 50 main codes represent more than 98%
ans = report_mod.groupby(['subject_code'],as_index = False).size().sort_values(ascending = False).head(50).sum()
print('The instances covered by the top 50 codes are: ' + str(ans))
print('Which represents: '+str(round((ans*100)/report_mod.shape[0]),) + '% of the total amount')

The instances covered by the top 50 codes are: 214151
Which represents: 98.0% of the total amount


In [20]:
report_mod.groupby(['subject_code'],as_index = False).size().sort_values(ascending = False).head(50)

subject_code
PH7     39249
PH1     38685
CPTH    23333
CPH3    16938
CPH4    14271
CPH2    10552
SPL      9573
PH14     6749
CFL      6023
SCVR     5987
GUL      3877
POW      3619
RWSG     3301
DCF3     2905
DCF2     2609
MSNG     2355
SUNK     2284
CCD1     1907
CRW2     1806
RICE     1525
CMUD     1409
CKD4     1183
DCF1     1101
COLL      922
CBAR      865
SPLE      836
RMK       817
FFL       692
CCS2      635
CFS1      569
CLLW      523
CCD3      522
DPD1      495
NZU       472
PDAM      456
DPD2      450
CCS3      447
NRSW      440
CRW1      436
HZB       399
CCD4      385
CCS4      345
SFL       330
DFL3      318
CSH1      311
DFL4      273
CKD3      260
HBW1      256
CKD2      229
CMD3      227
dtype: int64

In [107]:
#Keep only the instances that are within the scope
#CHECK HBW********
filter_subc = ['PH7','PH1','CPTH','CPH3','CPH4','CPH2','SPL','PH14','CFL','GUL',
               'DCF3','DCF2','CCD1','RICE','CMUD','DCF1','CBAR','SPLE',
              'CFS1','CCD3','DPD1','NZU','PDAM','DPD2','NRSW','CCD4',
               'SFL','CSH1','CKD3','CKD2','CMD3']

report_mod = report_mod[report_mod.subject_code.isin(filter_subc)]
report_mod = report_mod.reset_index(drop=True)
report_mod.shape

(198455, 15)

#### Priority Code

In [108]:
#Validate that all remaining priority codes are relevant
report_mod.groupby(['priority_code'], as_index = False).size().sort_values(ascending = False)

priority_code
H24    55203
D7     39249
H2     29133
D5     22481
D20    15001
D14     7022
dtype: int64

#### Enquiry Status Code

In [111]:
#Check the relevant enquiry status codes within the dataset
report_mod.groupby(['enq_status_code'], as_index = False).size().sort_values(ascending = False)

enq_status_code
400     131947
410      34341
420       9491
440       8336
480       4746
890       2927
430       2752
445       2628
450        632
435        325
490        121
460         58
240         51
470         43
210         23
245         13
250          6
1000         5
481          3
421          2
330          2
431          1
260          1
432          1
dtype: int64

In [110]:
#Standardise the enquiry status codes by removing aditional zeros.
report_mod['enq_status_code'] = report_mod['enq_status_code'].str.lstrip("0")

In [112]:
#Drop the enquiry status codes that are not within scope
filter_enq = ['440','100'] #Check 445, 435
report_mod = report_mod[~report_mod.enq_status_code.isin(filter_enq)]
report_mod = report_mod.reset_index(drop=True)
report_mod.shape

(190119, 15)

#### Log Effective Date  (**Optional**)

In [206]:
#Evaluate how many rows are included that don't have a date at all

report_mod['log_effective_date'].isnull().apply(lambda x: all(x), axis=1).sum()

21773

In [207]:
#Drop instances that have no date associated to the project
report_mod.dropna(subset=missing_date, how='all', inplace=True)
report_mod.shape

(27030, 36)

In [208]:
#Check the current stage for the remaining instances:
report_mod.groupby(report_mod['ProjectStage'], as_index = False).size().sort_values(ascending = False)

ProjectStage
Z     13349
6      6443
4a     1917
9      1882
5      1454
4c      812
X       606
2a      193
H       148
2c       98
2b       48
4b       24
1a       23
1b       20
3a        2
dtype: int64

In [209]:
#Drop instances that are in stages up to 4a and that were cancelled (X)
filter_stage = ['4a','2a','2c','X','2b','1a','1b']
report_mod = report_mod[~report_mod.ProjectStage.isin(filter_stage)]
report_mod = report_mod.reset_index(drop=True)
report_mod.shape

(24125, 36)

## Fill the blanks for rest of the categories with "Undefined"

In [210]:
report_mod['LatestWorksCost'].fillna(value = '0', inplace = True)
report_mod.fillna(value = 'Undefined', inplace = True)

In [211]:
#Use this to filter a specific value within a column
#report_mod.loc[report_mod['BudgetCode'] == 'ITP'].head(5)

### Save to a new CSV file 

In [212]:
report_mod.to_csv(r'C:\Users\J FernandezGomez\Jupyter Notebooks\Cleaning_Data_Notebooks\1_PMNET_IWReport\PMNET_processed_v2.csv')